In [1]:
import pandas as pd 
from collections import defaultdict
import copy 
import numpy as np 
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
df_origin = pd.read_excel(r'D:\Document\工作内容\1117社群与活跃关系探索\取数_可能影响活跃人数中社群人数占比的因素.xlsx')

In [3]:
df_origin.columns

Index(['comm_code', 'shop_name', 'name_org3', 'city', 'province',
       'shop_act_member_cnt_year', 'shop_group_act_member_cnt_year',
       'shop_group_member_cnt_year', 'group_cnt', 'add_unionid_cnt',
       'avg_change_cnt', 'new_cnt', 'old_18d_cnt', 'old_18d_rate',
       'old_19d_cnt', 'old_19d_rate', 'old_20d_cnt', 'old_20d_rate',
       'old_21d_cnt', 'old_21d_rate', 'old_22d_cnt', 'old_22d_rate',
       'old_23d_cnt', 'old_23d_rate', 'old_24d_cnt', 'old_24d_rate',
       'cur_open_diff_month', 'opennumber', 'act_group_rt'],
      dtype='object')

In [4]:
df = copy.deepcopy(df_origin)

# class_mapping = {label:idx for idx,label in enumerate(set(df['class label']))}
# df['class label'] = df['class label'].map(class_mapping)
# pd.get_dummies(df)  # one-hot 
class_mapping_1 = {label:idx for idx,label in enumerate(set(df['name_org3']))}
df['name_org3'] = df['name_org3'].map(class_mapping_1)
class_mapping_2 = {label:idx for idx,label in enumerate(set(df['city']))}
df['city'] = df['city'].map(class_mapping_2)
class_mapping_3 = {label:idx for idx,label in enumerate(set(df['province']))}
df['province'] = df['province'].map(class_mapping_3)



In [6]:
# df[['act_group_rt', 'group_cnt']].corr('pearson')
# dct_2
# df.groupby('group_cnt')['group_cnt'].count()
# df['group_cnt'].fillna(1, inplace=True)
# df['group_cnt'] = df['group_cnt'].astype(int)  # 修改数据类型
# df.loc[df['group_cnt'] == '\\N', 'group_cnt'] = 1  # 指定值替换
# dct_2
# df['group_cnt']

# 'old_18d_cnt', 'old_19d_cnt',
# 'old_20d_cnt', 'old_21d_cnt', 'old_22d_cnt', 'old_23d_cnt',
# 'old_24d_cnt'

# df['old_19_cnt'] = df['old_19d_cnt'] - df['old_18d_cnt']
# df['old_20_cnt'] = df['old_20d_cnt'] - df['old_19d_cnt']
# df['old_21_cnt'] = df['old_21d_cnt'] - df['old_20d_cnt']
# df['old_22_cnt'] = df['old_22d_cnt'] - df['old_21d_cnt']
# df['old_23_cnt'] = df['old_23d_cnt'] - df['old_22d_cnt']
# df['old_24_cnt'] = df['old_24d_cnt'] - df['old_23d_cnt']
df['old_24up_cnt'] = df['shop_act_member_cnt_year'] - df['old_24d_cnt']
df['old_24up_rate'] = df['old_24d_cnt'] / df['shop_act_member_cnt_year']
# df['act_group_rt'].dtype

In [7]:
# 计算相关性
# 三种相关系数使用条件
# 皮尔逊: 连续变量，同正态分布
# 斯皮尔曼: 非相同分布的变量
# 肯德尔: 有序分类变量

# 相关系数和p值: p值大于0.05表示没有显著性差异，即有相关性; 相关系数越大，p值越小; 

# 目标列  act_group_rt

# 属性特征
# 无序
# 斯皮尔曼
col_class = ['name_org3', 'city', 'province']
dct = defaultdict(int)
for i in col_class:
    dct[i] = df[[i, 'act_group_rt']].corr('spearman').iloc[0, 1]


# 数值特征
# 皮尔逊、斯皮尔曼
dct_2 = defaultdict(int)
# col_num =  ['shop_act_member_cnt_year', 'shop_group_act_member_cnt_year',
#             'shop_group_member_cnt_year', 'group_cnt', 'add_unionid_cnt',
#             'avg_change_cnt', 'new_cnt', 'old_18d_cnt', 'old_19d_cnt',
#             'old_20d_cnt', 'old_21d_cnt', 'old_22d_cnt', 'old_23d_cnt',
#             'old_24d_cnt', 'old_24up_cnt', 'cur_open_diff_month', 'opennumber']

col_num =  ['old_18d_rate', 'old_19d_rate',
            'old_20d_rate', 'old_21d_rate', 'old_22d_rate', 'old_23d_rate',
            'old_24d_rate', 'old_24up_rate', 'old_24up_rate']

for i in col_num:
    dct_2[i] = df[[i, 'act_group_rt']].corr('pearson').iloc[0, 1]


In [89]:
data = df[['name_org3', 'city', 'province', 'shop_act_member_cnt_year', 'shop_group_act_member_cnt_year',
            'shop_group_member_cnt_year', 'group_cnt', 'add_unionid_cnt',
            'avg_change_cnt', 'new_cnt', 'old_18d_cnt', 'old_19d_cnt',
            'old_20d_cnt', 'old_21d_cnt', 'old_22d_cnt', 'old_23d_cnt',
            'old_24d_cnt', 'old_24up_cnt', 'cur_open_diff_month', 'opennumber']]
data['label'] = np.select([df['act_group_rt']<=0.12, df['act_group_rt']>0.12], [0, 1])
# data.loc[df['act_group_rt']<=0.12, 'label'] = 0
# data.loc[df['act_group_rt']>0.12, 'label'] = 1

C:\Users\118252\AppData\Local\Temp/ipykernel_2148/2770680252.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label'] = np.select([df['act_group_rt']<=0.12, df['act_group_rt']>0.12], [0, 1])


In [93]:
# 构造训练数据和测试数据
X = data.drop(columns=['label'], axis=1)  
Y = data[['label']]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2, stratify=Y)
train_data = lgb.Dataset(data=x_train, label=y_train)
test_data = lgb.Dataset(data=x_test, label=y_test)

# 设置参数和模型训练
param = {
    'max_depth': 6
    , 'max_bin': 255
    , 'num_leaves': 31
    , 'learning_rate': 0.05
    , 'lambda_l1': 0.1
    , 'lambda_l2': 0.1
    , 'objective': 'binary'
    , 'boosting_type': 'gbdt'
    , 'metric': 'binary_logloss'
    # , 'early_stopping_round': 20
}
model = lgb.train(param, train_data, valid_sets=[test_data])

ypred = model.predict(x_test)

# auc
fpr, tpr, thresholds = metrics.roc_curve(y_test, ypred)  # thresholds阈值
print(metrics.auc(fpr, tpr))



[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 336, number of negative: 400
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000281 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3049
[LightGBM] [Info] Number of data points in the train set: 736, number of used features: 20
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.456522 -> initscore=-0.174353
[LightGBM] [Info] Start training from score -0.174353
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's binary_logloss: 0.650418
[Ligh

In [94]:
# 特征重要性
importance_list = model.feature_importance()  # importance_type='gain'
feature_list = model.feature_name()
feature_dict = {feature_list[i]: importance_list[i] for i in range(len(feature_list))}
result = sorted(feature_dict.items(), key=lambda x: x[1], reverse=True)
print(result)

[('shop_group_act_member_cnt_year', 534), ('shop_act_member_cnt_year', 376), ('old_24up_cnt', 127), ('old_24d_cnt', 51), ('new_cnt', 37), ('add_unionid_cnt', 34), ('shop_group_member_cnt_year', 29), ('avg_change_cnt', 25), ('cur_open_diff_month', 21), ('opennumber', 19), ('old_23d_cnt', 17), ('old_19d_cnt', 14), ('old_21d_cnt', 13), ('name_org3', 11), ('old_18d_cnt', 11), ('old_20d_cnt', 9), ('city', 5), ('province', 5), ('old_22d_cnt', 4), ('group_cnt', 1)]


In [8]:
dct_2

defaultdict(int,
            {'old_18d_rate': 0.1501257916439441,
             'old_19d_rate': 0.12569864689323318,
             'old_20d_rate': 0.11812084067645487,
             'old_21d_rate': 0.13425677451440962,
             'old_22d_rate': 0.15996446267381656,
             'old_23d_rate': 0.1747429217656335,
             'old_24d_rate': 0.1758565014536893,
             'old_24up_rate': 0.1758565014536893})